In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ImportError: cannot import name '__check_build' from 'sklearn' (C:\local\Anaconda3\lib\site-packages\sklearn\__init__.py)

In [ ]:
#Part1

In [ ]:
PATH = "./data/"


In [ ]:
data = pd.read_csv(os.path.join(PATH, 'titanic', 'train.csv')).set_index('PassengerId')


In [ ]:
data.head()


In [ ]:
data.columns
data['Sex']
male, female=list(data.Sex).count('male'),list(data.Sex).count('female')

male_survived=list(data.Sex[data.Survived==1]).count('male')
female_survived=list(data.Sex[data.Survived==1]).count('female')
print("Всего мужчин: ",male,"Всего женщин: ", female)
print("Выжило мужчин: ",male_survived,"Выжило женщин: ",female_survived)
print("Выжило мужчин в %: ",male_survived/male*100,"Выжило женщин в %: ",female_survived/female*100)

In [ ]:
#Из-за команды грузиться сначала женщинам их выжило больше

In [ ]:
survived , deceased =list(data.Survived).count(1),list(data.Survived).count(0)
print(' Выжившие= ',survived ,'\n','Умершие = ',deceased )

#Распределение возраста
age_distribution=data.hist(column=['Age'], bins=40, figsize=(16,8),label="Распределение возраста")
plt.title('Распределение возраста')
plt.legend()
plt.show()

age_mean_survived=data[(data['Survived']==1)]["Age"].mean()
age_median_survived=data[(data['Survived']==1)]["Age"].median()
age_mean_deceased=data[(data['Survived']==0)]["Age"].mean()
age_median_deceased=data[(data['Survived']==0)]["Age"].median()

print("Age_mean выживших = ",age_mean_survived)
print("Age_median выживших = ",age_median_survived)
print("Age_mean умерших = ",age_mean_deceased)
print("Age_median умерших = ",age_median_deceased)

#распределение возраста погибших и выживших
survived=data[(data['Survived']==1)]["Age"]
survived.hist(bins=40, figsize=(16,8), alpha=0.5,label="Выжившие люди")
deceased=data[(data['Survived']==0)]["Age"]
deceased.hist(bins=40, figsize=(16,8),alpha=0.6, label="Умершие люди")
plt.legend(loc='upper left')
plt.show()

In [3]:
#молодых грузили в шлюпки в последнюю очередь, поэтому и погибло больше всего людей от 16 до 46

In [4]:
#каюты первого класса были ближе к первой палубе + пассажиров первого класса начали раньше сажать в шлюпки

In [5]:
# распределение пассажиров по классам
class1_all=list(data.Pclass).count(1)
class2_all=list(data.Pclass).count(2)
class3_all=list(data.Pclass).count(3)
class1_survived=list(data.Pclass[data.Survived==1]).count(1)
class2_survived=list(data.Pclass[data.Survived==1]).count(2)
class3_survived=list(data.Pclass[data.Survived==1]).count(3)

# Распределение пассажиров по классам в %
print("1 класс выжившие: ", class1_survived, " from ",class1_all, " ( ",class1_survived/class1_all*100," %)" )
print("2 класс выжившие: ", class2_survived, " from ",class2_all, " ( ",class2_survived/class2_all*100," %)" )
print("3 класс выжившие: ", class3_survived, " from ",class3_all, " ( ",class3_survived/class3_all*100," %)" )

# Вступившие на борт пассажиры
survived_embarked=data[(data['Survived']==1)]["Embarked"]
deceased_embarked=data[(data['Survived']==0)]["Embarked"]
survived_embarked.hist(label="Выжившие")
deceased_embarked.hist(label="Умершие",alpha=0.5)
plt.legend()
plt.show()

# Пассажиры из разных портов
pas_embarked_S=list(data.Embarked).count("S")
pas_embarked_C=list(data.Embarked).count("C")
pas_embarked_Q=list(data.Embarked).count("Q")

#Выжившие пассажиры в %
percent_S=list(data.Embarked[data.Survived==1]).count("S")/list(data.Embarked).count("S")*100
percent_C=list(data.Embarked[data.Survived==1]).count("C")/list(data.Embarked).count("C")*100
percent_Q=list(data.Embarked[data.Survived==1]).count("Q")/list(data.Embarked).count("Q")*100

# Результаты
print(" Отправление из порта 'S': ",pas_embarked_S, " (", percent_S, "% выживших)")
print(" Отправление из порта 'C': ",pas_embarked_C," (", percent_C, "% выживших)")
print(" Отправление из порта 'Q': ",pas_embarked_Q, " (", percent_Q, "% выживших)")

NameError: name 'data' is not defined

In [ ]:
#Выжили пассажиры в основном 1ого и 2ого классов, тк они были ближе к палубе погрузки

In [ ]:
#Modelling

In [ ]:
data_missing_values=data.isna().sum()/data.isna().count()*100
print("% пробелов в признаках : ","\n",data_missing_values)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, RFECV, SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler


In [ ]:
data_test = pd.read_csv(os.path.join(PATH, 'titanic', 'test.csv'))


In [ ]:
y = data["Survived"]
X =data.drop(["Survived","Ticket","Cabin","Name","Fare"], axis=1)
X['Sex'] = X['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
X['Embarked']=X['Embarked'].fillna('Q')
X['Embarked'] = X['Embarked'].map( {'S': 0, 'C': 1,'Q':2} ).astype(int)
X['Age']=X['Age'].fillna(np.round(X['Age'].median(),0))

#X_test
X_test =data_test.drop(["Ticket","Cabin","Name","PassengerId","Fare"], axis=1)
X_test['Sex'] = X_test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
X_test['Embarked']=X_test['Embarked'].fillna('Q')
X_test['Embarked'] = X_test['Embarked'].map( {'S': 0, 'C': 1,'Q':2} ).astype(int)
X_test['Age']=X_test['Age'].fillna(np.round(X['Age'].median(),0))

# StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#print(X_scaled)
X_test_scaled=scaler.fit_transform(X_test)
#X_test_scaled

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

Y_train=y
X_train_scaled=X_scaled
logreg = LogisticRegressionCV(solver="lbfgs",cv=5, penalty="l2")
logreg.fit(X_train_scaled, Y_train)
Y_pred_log = logreg.predict(X_test_scaled)

acc_log = round(logreg.score(X_train_scaled, Y_train) * 100, 2)
print("Accuracy with LogReg (cv=5) = ",acc_log)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# KNN with scaling data
X_train=X_scaled
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test_scaled)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print("Accuracy with KNN = ",acc_knn)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": data_test["PassengerId"],
        "Survived": Y_pred_log
    })
submission.to_csv('./submission_final_titanic.csv', index=False)

In [ ]:
#Part2 - Porto Seguro’s Safe Driver Prediction

In [ ]:
insurance_data = pd.read_csv(os.path.join(PATH, 'porto', 'train.csv')).set_index('id')


In [ ]:
insurance_data.head()


In [ ]:
y = insurance_data['target']
X = insurance_data.drop(columns=['target'])

In [ ]:
numeric_features = [column for column in X if not (column.endswith("cat") or column.endswith("bin"))]
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_features = [column for column in X if (column.endswith("cat") or column.endswith("bin"))]
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14))])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
clf = pipeline.fit(X_train, y_train)


In [ ]:
#Мы получили результат модели - 0.96 Построим матрицу и проанализируем ошибки

In [ ]:
disp = plot_confusion_matrix(clf, X_valid, y_valid,
                             cmap=plt.cm.Blues,
                             normalize=None)

In [ ]:
#К сожалению наша модель полностью бесполезна

In [ ]:
insurance_data.describe()


In [ ]:
sns.countplot(x='target', data=insurance_data)
plt.show()


In [ ]:
# Сalc gini coeff
def gini(actual, pred, cmpcol = 0, sortcol = 1):  
    assert(len(actual) == len(pred))  
    epsilon = 1e-7
    values = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)  
    values = values[np.lexsort((values[:, 2], -1 * values[:, 1]))]  
    total = values[:, 0].sum() 
    gini_sum = (values[:, 0].cumsum().sum() + epsilon) / (total + epsilon)  
  
    gini_sum -= (len(actual) + 1) / 2  
    return gini_sum / len(actual)  
  
def gini_normalized(a, p):  
    '''Function to calculate the normalized gini coefficient'''
    return gini(a, p) / gini(a, a)

In [ ]:
y_pred = clf.predict_proba(X_valid)[:, 1]


In [ ]:
gini_normalized(y_valid, y_pred)
